## Modelado de datos para Funciones de Machine Learning

### Importando librerías 

In [2]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq


### Modelo de recomendación Item_Item

Un modelo de recomendación `item-item` basado en la similitud del coseno es un tipo de sistema de recomendación que sugiere elementos o ítems similares a aquellos que un usuario ha interactuado o mostrado interés previamente. Este enfoque se basa en la idea de que si a un usuario le gustó un cierto ítem, es probable que también le gusten ítems similares.
El perfil de un ítem generalmente se representa como un vector que contiene información relevante sobre ese ítem. Por ejemplo, en un sistema de recomendación de películas, el perfil de una película podría incluir información sobre el género, el director, el elenco de actores, las etiquetas asociadas, etc.

#### Se inicia con la creación de dummies , para luego crear la matriz necesaria para el  analisis de similitud de coseno

In [38]:
modelo_item= pd.read_parquet("data/modelo_item.parquet")
modelo_item.head(3)

,id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie


In [31]:
#Creacíon de dummies
modelo_item= pd.get_dummies(modelo_item, columns=["genres"], prefix="")

modelo_item= modelo_item.groupby(["id","app_name"]).sum().reset_index()

modelo_item.head(4)

,id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Calcular la similitud del coseno

Se calcula la similitud del coseno entre todos los pares posibles de ítems en función de sus perfiles. La similitud del coseno mide el ángulo entre dos vectores en un espacio multidimensional y varía de -1 (completamente opuestos) a 1 (idénticos). Cuanto mayor sea la similitud del coseno entre dos ítems, más parecidos se considerarán.

In [32]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])

In [33]:
similitudes.shape

(28850, 28850)

In [34]:
print(similitudes)

[[1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 ...
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [0.70710678 0.70710678 0.70710678 ... 0.70710678 1.         0.70710678]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]]


In [35]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtrar el juego de entrada por su ID
    juego_seleccionado = similitudes[modelo_item['id'] == id]
    
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcular la matriz de similitud coseno
    #similitudes = cosine_similarity(modelo_item.iloc[:,3:])
    
    # Obtener las puntuaciones de similitud del juego de entrada con otros juegos
    similarity_scores = similitudes[modelo_item[modelo_item['id'] == id].index[0]]
    
    # Obtener los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [37]:
recomendacion_juego("10")

AttributeError: 'numpy.ndarray' object has no attribute 'empty'

In [14]:
def top_game(game):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        None: Esta función imprime una lista de juegos 5 similares al dado.

    '''
    count = 1
    print('Similar games to {} include:\n'.format(game))
    for item in similitudes.sort_values(by = game, ascending = False).index[1:6]:
        print('No. {}: {}'.format(count, item))
        count +=1  

In [16]:
top_game('Counter_Strike')

Similar games to Counter_Strike include:



AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [8]:
def recomendacion_juego(genre):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        None: Un diccionario con 5 nombres de juegos recomendados.

    '''
    # Obtener la lista de juegos similares ordenados
    similar_games = similitudes.sort_values(by=genre, ascending=False).iloc[1:6]

    count = 1
    contador = 1
    recomendaciones = {}
    
    for item in similar_games:
        if contador <= 5:
            item = str(item)
            recomendaciones[count] = item
            count += 1
            contador += 1 
        else:
            break
    return recomendaciones

In [10]:
recomendacion_juego("genre_Adentures")

AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'